In [3]:
!apt update -qq;
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub;
!apt-get update -qq;
!apt-get install cuda gcc-5 g++-5 -y -qq;
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++;
!apt install cuda-8.0;


8 packages can be upgraded. Run 'apt list --upgradable' to see them.
--2018-10-29 20:29:29--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.189.146
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.189.146|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?jHj9pnkbXpPc8DhoBpTuzOHxQQQKT-xeUhbcavlp3Bt2Dt0pBm10TH3k85e1Ee9gUz8zF9PT15Be0L2SgA5MmnzZzNEMnKtB4uBd1JeM6U51VDf4MZyk7As-T0R7O_weyAG8Rf3FKpCl2xsI9CNrppSGtJW3ZLkE8HNUffvVUSZ-3BHzbXQ_ptyamry9qV7o4TByY4dm5D7ecZDlUXljC7dRow [following]
--2018-10-29 20:29:30--  https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?jHj9pnkbXpPc8DhoBpTuzOH

In [4]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Tue_Jun_12_23:07:04_CDT_2018
Cuda compilation tools, release 9.2, V9.2.148


In [5]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-8u0u_ytx
  Running setup.py bdist_wheel for NVCCPlugin ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-c07tqy8v/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [0]:
%load_ext nvcc_plugin

In [34]:
%%cu

#include<iostream>
using namespace std;

__global__ void maxi(int *a,int *r)
{
    int m=0;
    for(int i=0;i<100;i++)
    {
        if(m<a[i])m=a[i];
    }
    *r=m;
}

int main()
{
    int a[100],r;
    
    //a=(int*)malloc(100*sizeof(int));
    //r=(int*)malloc(sizeof(int));
    
    for(int i=0;i<100;i++)
    {
        a[i]=i+2;
    }
    
    int *d_a,*d_r;
    
    cudaMalloc((void**)&d_a,(100*sizeof(int)));
    cudaMalloc(&d_r,(sizeof(int)));
    
    cudaMemcpy(d_a,&a,(100*sizeof(int)),cudaMemcpyHostToDevice);
    cudaMemcpy(d_r,&r,(sizeof(int)),cudaMemcpyHostToDevice);
    
    maxi<<<1,1>>>(d_a,d_r);
    
    cudaMemcpy(&r,d_r,(sizeof(int)),cudaMemcpyDeviceToHost);
    cout<<r;
    return 0;
    
}

'101'

In [24]:
%%cu

/**
 * Copyright 1993-2012 NVIDIA Corporation.  All rights reserved.
 *
 * Please refer to the NVIDIA end user license agreement (EULA) associated
 * with this source code for terms and conditions that govern your use of
 * this software. Any use, reproduction, disclosure, or distribution of
 * this software and related documentation outside the terms of the EULA
 * is strictly prohibited.
 */
#include <stdio.h>
#include<cuda.h>
#include <stdlib.h>
#include<time.h>

#define SIZE 6

__global__ void max(int *a, int *c)
{
	int m=0;
    for(int i=0;i<SIZE;i++)
    {
        if(m<a[i])m=a[i];
    }
    *c=m;
}

__global__ void min(int *a, int *c)
{
	int m=9999;
    for(int i=0;i<SIZE;i++)
    {
        if(m>a[i])m=a[i];
        
    }
    *c=m;
}

__global__ void calcavg(int *a, float *avg)
{
	*avg=0;
	for(int i=0;i<SIZE;i++)
	{
		*avg=*avg+a[i];
	}
	*avg=*avg/SIZE;
}


int main(void)
{
	int i;
	srand(time(NULL));
	int a[SIZE];
	int c,d;
	float avg;

	int *dev_a,*dev_c,*dev_d;
	float *average;

	cudaMalloc((void **)&dev_a, SIZE*sizeof(int));
	cudaMalloc(&dev_c, sizeof(int));
	cudaMalloc(&dev_d, sizeof(int));
	cudaMalloc(&average, sizeof(float));

	for(i=0;i<SIZE;i++)
	{
		a[i] = rand()%20+1;
	}

	printf("\nThe array is:\n");
	for(i=0;i<SIZE;i++)
	{
		printf("%d  ",a[i]);
	}


	cudaMemcpy(dev_a,a,sizeof(a),cudaMemcpyHostToDevice);
	max<<<1,SIZE>>>(dev_a,dev_c);
	min<<<1,SIZE>>>(dev_a,dev_d);
	calcavg<<<1,1>>>(dev_a,average);
	cudaMemcpy(&c,dev_c,sizeof(int),cudaMemcpyDeviceToHost);
	cudaMemcpy(&d,dev_d,sizeof(int),cudaMemcpyDeviceToHost);
	cudaMemcpy(&avg,average,sizeof(float),cudaMemcpyDeviceToHost);

	printf("\nMax is: %d",c);
	printf("\nMin is: %d",d);
	printf("\nAverage is: %f",avg);
	cudaFree(dev_a);
	cudaFree(dev_c);
	cudaFree(dev_d);
	cudaFree(average);
	return 0;
}


'\nThe array is:\n8  8  13  13  6  20  \nMax is: 20\nMin is: 6\nAverage is: 11.333333'

In [9]:
%%cu

#include<iostream>
#include<cuda.h>
#include<bits/stdc++.h>
#include<stdlib.h>
#include<stdio.h>
#define SIZE 50
using namespace std;


__global__ void maxi(int *a,int *c)
{
    int m=0;
    for(int i=0;i<SIZE;i++)
    //int i=threadIdx.x;
    {
        if(m<a[i])m=a[i];
    }
    *c=m;
}

__global__ void mini(int *a,int *c)
{
    int m=999;
    for(int i=0;i<SIZE;i++)
    {
        if(m>a[i])m=a[i];
    }
    *c=m;
}

__global__ void aver(int *a,float *c)
{
    float m=0.0;
    
    for(int i=0;i<SIZE;i++)
    {
        m+=a[i];
    }
    m=m/SIZE;
    
    *c=m;
}


int main()
{
    int a[SIZE],min,max;
    float avg,time;
    
    int *d_a,*d_res;
    float *d_avg;
    
    for(int i=0;i<SIZE;i++)
    {
        a[i]=rand()%100;
    }
    
    cudaEvent_t start,stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start,0);
    
    cudaMalloc((void**)&d_a,SIZE*sizeof(int));
    cudaMalloc(&d_res,sizeof(int));
    cudaMalloc(&d_avg,sizeof(float));
    
    cudaMemcpy(d_a,a,sizeof(a),cudaMemcpyHostToDevice);
    //cudaMemcpy(d_res,&c,sizeof(int,cudaMemcpyHostToDevice));
    
    maxi<<<1,SIZE>>>(d_a,d_res);
    
    cudaMemcpy(&max,d_res,sizeof(int),cudaMemcpyDeviceToHost);
    
    mini<<<1,SIZE>>>(d_a,d_res);
    
    cudaMemcpy(&min,d_res,sizeof(int),cudaMemcpyDeviceToHost);
    
    aver<<<1,1>>>(d_a,d_avg);
    
    cudaMemcpy(&avg,d_avg,sizeof(float),cudaMemcpyDeviceToHost);
    
    cudaEventRecord(stop,0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&time,start,stop);
    
    cout<<"Array is \n";
    for(int i=0;i<SIZE;i++)
    {
        cout<<a[i]<<"  ";
        
    }
    
    cout<<endl<<"Max is: "<<max;
    cout<<endl<<"Min is: "<<min;
    cout<<endl<<"Everage is: "<<avg;
    cout<<endl<<"Time elapsed is: "<<time;
    
    return 0;
    

}

'Array is \n83  86  77  15  93  35  86  92  49  21  62  27  90  59  63  26  40  26  72  36  11  68  67  29  82  30  62  23  67  35  29  2  22  58  69  67  93  56  11  42  29  73  21  19  84  37  98  24  15  70  \nMax is: 98\nMin is: 2\nEverage is: 50.62\nTime elapsed is: 0.44976'